# Part I - reading postal codes (scroll down for tasks 2 and 3)


Import all necessary packages

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Download the page into parser

In [ ]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup (r.text, "html.parser")

Parse page into a dictionary where each postal code is represented with one line

In [ ]:
NA  ="Not assigned" # A constant for "Not assigned marker"

pc_dict = dict() # A dictionary for postal codes. One postal code - one entry in the dict.

rows = soup.table.find_all("tr") # The data are in the first table. Let's read line by line
for r in rows:
    tds = r.find_all("td")
    if len(tds) > 0:                     # ignore headings
        code = tds[0].text.strip()       # remove extra characters (there are some!)
        borough = tds[1].text.strip()
        neigh = tds[2].text.strip()
        
        if (borough != NA):              # Ignore lines where borough is unassigned
            if (neigh == NA):            # Fill in unassigned neighbourhood
                neigh = borough
            if code in pc_dict:          # Did we see this postal code before?
                line = pc_dict[code]     # ... yes - then just add another neighbourhood
                line[2] += "," + neigh
            else:                        # ... no - create a new entry
                pc_dict[code] = [code, borough, neigh]
#print(pc_dict)

Transform dictionary into dataframe

In [ ]:
postalCodes = pd.DataFrame(list(pc_dict.values()), columns = ['PostalCode', 'Borough', 'Neighborhood'])
postalCodes.head()

In [ ]:
postalCodes.shape

# Part 2 - Get coordinates

Time to get coordinates for each postal code.

First, let's try geocoder as in the assignment description

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
inf_loop_braker = 0
while(lat_lng_coords is None and inf_loop_braker < 5):
    g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
    lat_lng_coords = g.latlng
    print ("Iteration: {}, coordinates: {}".format(inf_loop_braker, lat_lng_coords))
    inf_loop_braker += 1


No matter gow many time I try and what address I use it is always None

Then, let's try Nominatim as in the lab

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ny_explorer")
def test_address(address):
    location = geolocator.geocode(address)
    if location is None:
        print ("Location '{}' not found".format(address))
    else:
        print ("Coordinates for '{}': {}, {}".format(address, location.latitude, location.longitude))

In [ ]:
test_address("M5A, Toronto, Ontario")
test_address("M5A, Toronto, Ontario, Canada")
test_address("M5G, Toronto, Ontario")
test_address("M5G, Toronto, Ontario, Canada")

In [ ]:
n_filled = 0
for i in postalCodes.index:
    address = "{}, Toronto, Ontario".format(postalCodes.PostalCode[i])
    location = geolocator.geocode(address)
    if not location is None:
        n_filled += 1
        
print ("Number of found zip codes: {}".format(n_filled))

Some zip codes are recogined, but only minor part. And even for those recognized coordinates are different than in "Geospacial_coordinates.csv".

Thus for consistency let's use Geospacial_coordinates.csv

In [ ]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

In [ ]:
postalCodes = postalCodes.merge(coordinates, left_on="PostalCode", right_on="Postal Code").drop(columns=["Postal Code"])
postalCodes.head()

# Part 3, explore, visualize and cluster

### Let's start with simply showing all the neighborhoods on the map

In [ ]:
import folium

# Create a map centered at Toronto
location = geolocator.geocode("Toronto, Canada")
map_toronto = folium.Map(location=[location.latitude, location.longitude], zoom_start=11)

# Select only boroughs with "Toronto" in the name
torontoCodes = postalCodes[postalCodes.Borough.apply(lambda x: "Toronto" in x)]

# Add markers for each Neighborhood (or more precisely - for each postal code)
for lat, lng, borough, neighborhood in zip(torontoCodes['Latitude'], torontoCodes['Longitude'], torontoCodes['Borough'], torontoCodes['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Venues data

Now let get venues from FourSquare. **In order to run further cells you need to  type in your credentials in the below box or create a file with Foursquare ID and secret.**

In [ ]:
CLIENT_ID = 'REPLACE WITH YOUR ID OR CREATE A FILE FS_cred.txt' # your Foursquare ID
CLIENT_SECRET = 'REPLACE WITH YOUR SECRET OR CREATE A FILE FS_cred.txt' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


try:
    filename = "FS_cred.txt" 
    # This block will read 
    with open(filename, "r") as f:
        lines = f.readlines()
        CLIENT_ID = lines[0].strip()
        CLIENT_SECRET = lines[1].strip()
        print ("Foursquare credentials read from '{}'".format(filename))
except:
    print ("Could not read '{}'".format(filename))

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Get venues around "zipcode center" location

In [ ]:
LIMIT = 100
def getNearbyVenues(codes, names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for code, name, lat, lng in zip(codes, names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zipcode', 
                  'Neighborhood',           
                  'Zipcode Latitude', 
                  'Zipcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(  codes=torontoCodes['PostalCode'],
                                   names=torontoCodes['Neighborhood'],
                                   latitudes=torontoCodes['Latitude'],
                                   longitudes=torontoCodes['Longitude']
                                  )


In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In most cases number of venues is below the limit of 100. Several neighborhood have less than 10 venues

In [ ]:
toronto_venue_counts = toronto_venues.groupby('Neighborhood').count().reset_index()[["Neighborhood", "Venue"]]
toronto_venue_counts.columns = ["Neighborhood", "N of Venues"]
toronto_venue_counts

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In the following cells neighborhood will be clustered by ratio of various venue categories. 

Convert categorical variable into one-hot encoding and then to ratio of each category within a neighborhood:

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Create a dataframe that shows 10 most common categories:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Perform clustering

In [ ]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = torontoCodes

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(toronto_venue_counts.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

Visualize clustering

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As we see on the map, clustering did not produce any significant grouping. Most of points went to cluster 0. Other clusters are formed from the neighborhoods witrh only 1-4 venues. Changing target number of clusters does not help.

A hypothesis that could be tested as a next step: 
* group categories into some super-categories, such as drink/food, shops, parks, other
* cluster by number of national cuisine restaurants
* number or venues may be one best single criteria for clustering

Choice of hypothesis to test next would depend on task context (if it was real world task)

In [ ]:
toronto_merged.groupby("Cluster Labels").count().reset_index()[["Cluster Labels", "PostalCode"]]

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] != 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]